In [ ]:
!pip install langchain openai chromadb requests beautifulsoup4 python-dotenv langchain_community tiktoken langchain_groq

In [6]:
import requests
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from google.colab import userdata

load_dotenv()
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')


def fetch_news():
    url = f"https://newsapi.org/v2/top-headlines?country=us&category=technology&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get("articles", [])
        news_text = "\n".join([article["title"] + ": " + article["description"] for article in articles if article["description"]])
        return news_text
    return "Failed to fetch live news."

news_text = fetch_news()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(news_text)


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma.from_texts(chunks, embedding_model)

llm = ChatGroq(model_name="mixtral-8x7b-32768", groq_api_key=GROQ_API_KEY)

retriever = vector_db.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

while True:
    query = input("\nEnter Your Question (or type 'exit' to quit): ")
    if query.lower() == "exit":
        print("Exiting... 👋")
        break
    response = qa_chain.run(query)
    print("\nAnswer:\n", response)



Enter Your Question (or type 'exit' to quit): news about iphone

Answer:
 The user's question: "Is there any news about the iPhone 16e or a new data center on the moon?"

Yes, there is news about the iPhone 16e. Apple has released its latest budget handset, the $599 iPhone 16e, without any big event or media coverage. The iPhone 16e is equipped with an A18 chip and Apple Intelligence.

There is also news about a new data center on the moon. Lonestar and Phison are sending the infrastructure needed to build out a data center to the moon with the goal of growing its capacity to a petabyte in the future.

There is no news about any connection between the iPhone 16e and the new data center on the moon.

Enter Your Question (or type 'exit' to quit): exit
Exiting... 👋
